<a href="https://colab.research.google.com/github/labtest0018/test/blob/main/Optimization_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import keras as keras
from tensorflow.keras.layers import Dense,Dropout,Conv2D,Flatten,MaxPooling2D,Activation,BatchNormalization, Input
import tensorflow as tf
from matplotlib.patches import Ellipse

In [ ]:
%matplotlib inline

In [ ]:
dim=6 #Number of rotators
freq_min=2000 #Starting Frequency
freq_max=7000 #Final frequency
freq_step=100 #Frequency step
freq_points=(freq_max-freq_min)//freq_step+1
freq=np.linspace(freq_min, freq_max, freq_points) #list of frequencies

In [ ]:
#FORWARD MODEL ONE FREQ
model=tf.keras.Sequential()
model.add(Dense(units=50, input_shape=(dim,)))
model.add(Dense(50, activation="relu", kernel_initializer="he_normal"))
model.add(Dense(20, activation="relu", kernel_initializer="he_normal"))
model.add(Dense(1))
model.add(Activation('linear'))
optimizer=keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='mse')

In [ ]:
model.load_weights("/content/drive/MyDrive/Optimization/virtual_oneoutput_4000Hz_v1.h5")


In [ ]:
model.summary()

In [ ]:
#FORWARD MODEL Spectra
model2=tf.keras.Sequential()
model2.add(Dense(units=50, input_shape=(dim,)))
model2.add(Dense(50, activation="relu", kernel_initializer="he_normal"))
model2.add(Dense(20, activation="relu", kernel_initializer="he_normal"))
model2.add(Dense(freq_points))
model2.add(Activation('linear'))
optimizer2=keras.optimizers.Adam(learning_rate=0.001)
model2.compile(optimizer=optimizer2, loss='mse')


In [ ]:
model2.load_weights("/content/drive/MyDrive/Optimization/virtual_rangeoutput_v1.h5")

In [ ]:
#Functions to calculate transmission value/spectra for the desired position of the rotators
#with the help of pre-trained NNs
def acoustic1freq(rot, model):
  res=model.predict(rot.reshape(1,-1))
  #res=model.predict(rot)
  res2=res[0][0]
  return res2

def acousticRange(rot, model):
  res=model2.predict(rot.reshape(1,-1))
  res2=res[0]
  return res2


In [ ]:
rot_position=np.array([80,10,150,100,20,50]) #Note that you should use numpy array and not the list
one_val=acoustic1freq(rot_position, model=model)
spectra=acousticRange(rot_position, model=model2)
print (one_val)
print (spectra)
print (freq)

In [ ]:
#Example of drawing
fig = plt.figure()
ax1 = fig.add_subplot(111)
ax1.set_xlabel('Amplitude')
ax1.set_ylabel('Frequency')
ax1.set_title('Transmission spectra')
#ax1.set_title('Whole normal dist')
ax1.plot(spectra, freq)

In [ ]:
#Functions to draw the position of rotators
def setupconfig():
  fig_el2 = plt.figure()
  ax1_el2 = fig_el2.add_subplot(211)
  #ax2 = fig.add_subplot(212)
  ax1_el2.set_xlim(0,35)
  ax1_el2.set_ylim(-2.5,2.5)
  ax1_el2.set_aspect('equal')
  #ax2.set_xlim(0,35)
  #ax2.set_ylim(-2.5,2.5)
  #ax2.set_aspect('equal')
  ells2 = [Ellipse(xy=((i+1)*5,0), width=4, height=1, angle=0)
          for i in range(6)]
  for e in ells2:
    ax1_el2.add_artist(e)
  return ells2, fig_el2, ax1_el2

def showconfig(rot,ells2, fig_in, ax_in):
  #fig_out=fig_in
  for i in range(len(ells2)):
    ells2[i].angle=rot[i]
  fig_in.canvas.draw()
  fig_in.canvas.flush_events()
  #display(fig_in)

def showconfigMain(rot):
  ells_in, fig_in, ax_in=setupconfig()
  showconfig(rot,ells_in, fig_in, ax_in)


In [ ]:
#Example of position drawing
showconfigMain(rot_position)

In [ ]:
rot_position


array([ 80,  10, 150, 100,  20,  50])